In [7]:
import os
import shutil
from sklearn.model_selection import train_test_split

def is_valid_file(file_path):
    return os.path.isfile(file_path)

data_dir = 'yaogan/datasets'
classes = ['airplane', 'bridge', 'palace', 'ship', 'stadium']

train_dir = 'yaogan/train'
test_dir = 'yaogan/test'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)
    file_names = [f for f in os.listdir(os.path.join(data_dir, cls)) if is_valid_file(os.path.join(data_dir, cls, f))]
    train_files, test_files = train_test_split(file_names, test_size=0.1, random_state=42)
    
    for file in train_files:
        shutil.copy(os.path.join(data_dir, cls, file), os.path.join(train_dir, cls, file))
        
    for file in test_files:
        shutil.copy(os.path.join(data_dir, cls, file), os.path.join(test_dir, cls, file))

In [6]:
!pip install scikit-learn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 2.0 MB/s eta 0:00:0000:0100:01


In [8]:
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import functional as F
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.optim.lr_scheduler import StepLR
# 数据增强
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(180),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
        transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.4), ratio=(0.2, 4.0), value='random')
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join('yaogan', x), data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

# 使用预训练的EfficientNet B7
model = models.efficientnet_b7(pretrained=True)

# 修改分类层
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(class_names))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 定义损失函数
cross_entropy_loss = nn.CrossEntropyLoss()
label_smoothing_loss = LabelSmoothingLoss(classes=len(class_names), smoothing=0.1)  # 这里的smoothing参数可以根据需要调整

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 学习率调度器
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# 定义损失函数的加权组合
def combined_loss(pred, target, alpha=0.9):
    ce_loss = cross_entropy_loss(pred, target)
    ls_loss = label_smoothing_loss(pred, target)
    return alpha * ce_loss + (1 - alpha) * ls_loss

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # 每个epoch都有训练和验证阶段
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # 设置模型为训练模式
            else:
                model.eval()   # 设置模型为评估模式

            running_loss = 0.0
            running_corrects = 0

            # 遍历数据
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # 前向传播
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 仅在训练阶段进行反向传播和优化
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# 设置训练的epoch数
num_epochs = 25

# 训练模型
model = train_model(model, combined_loss, optimizer, scheduler, num_epochs=num_epochs)

# 保存模型
torch.save(model.state_dict(), 'efficientnet_b7_yaogan.pth')

/root/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B7_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B7_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-c5b4e57e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-c5b4e57e.pth
100%|██████████| 255M/255M [16:58<00:00, 262kB/s] 


Epoch 0/24
----------


100%|██████████| 1682/1682 [07:52<00:00,  3.56it/s]


train Loss: 0.3044 Acc: 0.9180


100%|██████████| 187/187 [00:15<00:00, 12.35it/s]


...

test Loss: 0.0710 Acc: 0.9998
Epoch 24/24
----------


100%|██████████| 1682/1682 [07:53<00:00,  3.55it/s]


train Loss: 0.0822 Acc: 0.9961


100%|██████████| 187/187 [00:15<00:00, 12.18it/s]


test Loss: 0.0710 Acc: 0.9998


In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn.functional as F

# 数据增强
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(180),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
        transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.4), ratio=(0.2, 4.0), value='random')
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join('yaogan', x), data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

# 定义教师模型
teacher_model = models.efficientnet_b7(pretrained=False)
num_ftrs = teacher_model.classifier[1].in_features
teacher_model.classifier[1] = nn.Linear(num_ftrs, len(class_names))
teacher_model.load_state_dict(torch.load('efficientnet_b7_yaogan.pth'))

# 定义学生模型
student_model = models.efficientnet_b0(pretrained=True)
num_ftrs = student_model.classifier[1].in_features
student_model.classifier[1] = nn.Linear(num_ftrs, len(class_names))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
teacher_model = teacher_model.to(device)
student_model = student_model.to(device)

# 定义损失函数
cross_entropy_loss = nn.CrossEntropyLoss()

# 定义优化器
optimizer = optim.Adam(student_model.parameters(), lr=0.001)

# 学习率调度器
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# 定义蒸馏损失函数
def distillation_loss(student_logits, teacher_logits, labels, temperature=2.0, alpha=0.9):
    soft_loss = F.kl_div(F.log_softmax(student_logits / temperature, dim=1),
                         F.softmax(teacher_logits / temperature, dim=1),
                         reduction='batchmean') * (temperature ** 2)
    hard_loss = cross_entropy_loss(student_logits, labels)
    return alpha * hard_loss + (1 - alpha) * soft_loss

def train_model(teacher_model, student_model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # 每个epoch都有训练和验证阶段
        for phase in ['train', 'test']:
            if phase == 'train':
                student_model.train()  # 设置模型为训练模式
            else:
                student_model.eval()   # 设置模型为评估模式

            running_loss = 0.0
            running_corrects = 0

            # 遍历数据
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # 前向传播
                with torch.set_grad_enabled(phase == 'train'):
                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs)
                    student_outputs = student_model(inputs)
                    _, preds = torch.max(student_outputs, 1)
                    loss = criterion(student_outputs, teacher_outputs, labels)

                    # 仅在训练阶段进行反向传播和优化
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # 每轮结束时保存模型权重
        torch.save(student_model.state_dict(), f'efficientnet_b0_yaogan_distilled_epoch_{epoch}.pth')

    return student_model

# 设置训练的epoch数
num_epochs = 25

# 训练学生模型
student_model = train_model(teacher_model, student_model, distillation_loss, optimizer, scheduler, num_epochs=num_epochs)

# 保存最终的学生模型
torch.save(student_model.state_dict(), 'efficientnet_b0_yaogan_distilled_final.pth')

/root/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/root/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [01:23<00:00, 257kB/s] 


Epoch 0/24
----------


100%|██████████| 1682/1682 [03:37<00:00,  7.74it/s]


train Loss: 0.2696 Acc: 0.9231


100%|██████████| 187/187 [00:18<00:00, 10.12it/s]


...

test Loss: 0.0170 Acc: 0.9997
Epoch 24/24
----------


100%|██████████| 1682/1682 [03:38<00:00,  7.71it/s]


train Loss: 0.0264 Acc: 0.9948


100%|██████████| 187/187 [00:18<00:00, 10.13it/s]

test Loss: 0.0179 Acc: 0.9997
